# Boltz with Restraint-Guided Inference (boltz_restr)

This notebook runs **boltz_restr**, an extended version of Boltz with restraint-guided inference.

### Key Features
- **Conformer restraints**: 100% chirality reproduction for ligands.
- **Distance restraints**: Sampling domain motion, dissociation pathways, and improving docking poses.

--- 
**Citation:**
If you use this work, please cite:
- *Improving Stereochemical Limitations in Protein-Ligand Complex Structure Prediction*, Ishitani, R. and Moriwaki, Y., ACS Omega (2025).
- [Original Boltz Papers](https://github.com/jwohlwend/boltz)

**Repository:** [github.com/cddlab/boltz_restr](https://github.com/cddlab/boltz_restr)

In [ ]:
#@title 1. Install Dependencies
#@markdown This step installs `boltz_restr` and necessary libraries. It may take a few minutes.

import os

# Install boltz_restr from GitHub
print("Installing boltz_restr...")
!uv venv --allow-existing
!uv pip install "boltz[cuda] @ git+https://github.com/cddlab/boltz_restr.git"

# Install compatible torch-cluster
print("Installing torch-cluster...")
import torch
TORCH = "2.8.0"
CUDA = 'cu' + torch.version.cuda.replace('.', '')
!uv pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!uv pip uninstall torchvision

print("Installation complete.")

In [ ]:
%%writefile job.yaml
#@title 2. Edit Input Configuration (YAML)
#@markdown Edit the YAML content below directly to define your protein/ligand sequences and restraints.
#@markdown By default, this is set to the example provided in the README.

sequences:
  - protein:
      id: A
      sequence: ASWSHPQFEKIEGRDTWRYAFEEAMTDVQGVYAQKFKEEIEANSDHEIQLFPYGTLGESADIMEQTQDGILQFVDQSPGFTGSLIPEAQVFFVPYLLPTDQDHLARFFKESKAINDMFKPLYADQGLELLNMFPEGEVAMTTKTPVTTCSDLDEVKFRVMTNPLLVESYKAFGATPTPLPWGEVYGGLQTNVIQGQENPTFFLYSTKIYEVTDYITYAGHNNFTTAVMANKDFYDGLSAEDQQLVQNAALAAYDHTVVYQQQAADTELAKIMEAKPEMQVTVLTDEQRSCFKEAAAEVEAKFIEMTGDSGAAILKQMKADLAATAN
  - ligand:
      id: B
      ccd: 4CS
      conformer_restraints: true

restraints_config:
  verbose: true
  max_iter: 1000
  method: "CG"
  start_sigma: 1.0
  gpu: true
  # see README.md in boltz_restr for using distance restraints
  # distance_restraints_config:
  #   - atom_selection1: "chain A"
  #     atom_selection2: "chain B"
  #     harmonic:
  #       target_distance: 50.0
  conformer_restraints_config:
    bond:
      weight: 1
    angle:
      weight: 1
    chiral:
      weight: 1
    vdw:
      weight: 1

In [ ]:
#@title 3. Run Prediction

!uv run boltz predict job.yaml --out_dir boltz_results_job --use_msa_server --write_full_pae --write_full_pde --save_intermediate_steps --seed 0 --model boltz2

In [ ]:
#@title 4. Download Results
#@markdown Zips the output folder and triggers a download.
import shutil
import os
from google.colab import files

output_dir = "/content/boltz_results_job"
zip_name = "results.zip"

if os.path.exists(output_dir):
    shutil.make_archive(zip_name.replace('.zip', ''), 'zip', output_dir)
    files.download(zip_name)
    print(f"Zipped {output_dir} to {zip_name} and started download.")
else:
    print(f"Error: Output directory '{output_dir}' not found. Please check if the prediction ran successfully.")